# Lab | Tools prompting

**Replace the existing two tools decorators, by creating 3 new ones and adjust the prompts accordingly**

### How to add ad-hoc tool calling capability to LLMs and Chat Models

:::{.callout-caution}

Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide for more information.

In this guide, we'll see how to add **ad-hoc** tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools. Here's a diagram of the logic:

<br>

![chain](https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/tool_chain.svg)

## Setup

We'll need to install the following packages:

In [1]:
!pip install --upgrade --quiet langchain langchain-community

If you'd like to use LangSmith, uncomment the below:

In [1]:
import getpass
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

You can select any of the given models for this how-to guide. Keep in mind that most of these models already [support native tool calling](https://python.langchain.com/docs/integrations/chat), so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

To illustrate the idea, we'll use `phi3` via Ollama, which does **NOT** have native support for tool calling. If you'd like to use `Ollama` as well follow [these instructions](https://python.langchain.com/docs/integrations/chat/ollama).

In [ ]:
!pip install langchain-ollama

In [2]:
from langchain_community.llms import Ollama
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="phi3")
#model = Ollama(model="phi3")

## Create a tool

First, let's create an `add` and `multiply` tools. For more information on creating custom tools, please see [this guide](https://python.langchain.com/docs/how_to/custom_tools/).

In [26]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}


In [27]:
multiply.invoke({"x": 4, "y": 5})

20.0

## Creating our prompt

We'll want to write a prompt that specifies the tools the model has access to, the arguments to those tools, and the desired output format of the model. In this case we'll instruct it to output a JSON blob of the form `{"name": "...", "arguments": {...}}`.

In [28]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(x: float, y: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.


In [29]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [30]:
chain = prompt | model
message = chain.invoke({"input": "what's 3 plus 1132"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

```json
{
  "name": "add",
  "arguments": {
    "x": 3,
    "y": 1132
  }
}
```

The add tool is used because we're adding two numbers together. Here are the values for each argument in our case:
- `x`: The first number to be added (smaller or less significant). In this example, it's 3.
- `y`: The second number to be added (larger or more significant). Here, it'd be 1132. Since the add function takes integers as inputs and we have a decimal point in one of our numbers here I will convert that into an integer by taking only its integral part before adding which would give us `add(x: 3 , y: 1130)` or else, if you want to consider just the number without rounding off then it can be done as add(x:3.0,y: 1132) where we are considering decimal part also in our input and treating them as a float so that they don't affect addition of two numbers here



## Adding an output parser

We'll use the `JsonOutputParser` for parsing our models output to JSON.

In [31]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

{'name': 'multiply', 'arguments': {'x': 13, 'y': 4}}

:::{.callout-important}

🎉 Amazing! 🎉 We now instructed our model on how to **request** that a tool be invoked.

Now, let's create some logic to actually run the tool!
:::

## Invoking the tool 🏃

Now that the model can request that a tool be invoked, we need to write a function that can actually invoke 
the tool.

The function will select the appropriate tool by name, and pass to it the arguments chosen by the model.

In [32]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

Let's test this out 🧪!

In [33]:
invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})

15.0

## Let's put it together

Let's put it together into a chain that creates a calculator with add and multiplication capabilities.

In [34]:
chain = prompt | model | JsonOutputParser() | invoke_tool
chain.invoke({"input": "what's thirteen times 4.14137281"})

53.83784653

## Returning tool inputs

It can be helpful to return not only tool outputs but also tool inputs. We can easily do this with LCEL by `RunnablePassthrough.assign`-ing the tool output. This will take whatever the input is to the RunnablePassrthrough components (assumed to be a dictionary) and add a key to it while still passing through everything that's currently in the input:

In [35]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'multiply',
 'arguments': {'x': 13, 'y': 4.14137281},
 'output': 53.83784653}

## What's next?

This how-to guide shows the "happy path" when the model correctly outputs all the required tool information.

In reality, if you're using more complex tools, you will start encountering errors from the model, especially for models that have not been fine tuned for tool calling and for less capable models.

You will need to be prepared to add strategies to improve the output from the model; e.g.,

1. Provide few shot examples.
2. Add error handling (e.g., catch the exception and feed it back to the LLM to ask it to correct its previous output).

# **Enhanced Tool Invoker with Few-shot Examples and Error Handling**

**Ceeating tools**

In [3]:
from langchain_core.tools import tool

@tool
def add(x: int, y: int) -> int:
    """Add two numbers."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y

@tool
def subtract(x: int, y: int) -> int:
    """Subtract two numbers."""
    return x - y

@tool
def divide(x: float, y: float) -> float:
    """Divide two numbers. Returns an error for division by zero."""
    if y == 0:
        raise ValueError("Division by zero is not allowed.")
    return x / y

@tool
def power(base: float, exponent: float) -> float:
    """Raise a number to a power."""
    return base ** exponent

# Collect all tools
my_tools = [add, multiply, subtract, divide, power]

# Let's inspect the tools
for t in my_tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}
--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
subtract
Subtract two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}
--
divide
Divide two numbers. Returns an error for division by zero.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
power
Raise a number to a power.
{'base': {'title': 'Base', 'type': 'number'}, 'exponent': {'title': 'Exponent', 'type': 'number'}}


**Prompt creation using few-shot examples to guide the model**

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(my_tools)
print(rendered_tools)

add(x: int, y: int) -> int - Add two numbers.
multiply(x: float, y: float) -> float - Multiply two numbers together.
subtract(x: int, y: int) -> int - Subtract two numbers.
divide(x: float, y: float) -> float - Divide two numbers. Returns an error for division by zero.
power(base: float, exponent: float) -> float - Raise a number to a power.


**Prompt Design No. 01 - Not Used - too much details**

system_prompt = """\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:
""" + rendered_tools + """

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.

FOLLOW the ORDER of OPERATIONS: Parentheses, Exponents, Multiplication/Division, Addition/Subtraction.

When answering:
1. Return all operations in JSON format, step by step.
2. Execute operations sequentially, passing intermediate results where necessary.
3. For each step, include the tool name, arguments, and intermediate result.

Here are some example inputs and the correspinding tools to call:

###Example SINGLE-STEP Input:
Input: What is 3 plus 5?
function_call: {{"name": "add", "arguments": {{"x": 3, "y": 5}}, "result": 8}}

Input: Multiply 4 by 6
function_call: {{"name": "multiply", "arguments": {{"x": 4, "y": 6}}, "result": 24}}

Input: What is 10 minus 4?
function_call: {{"name": "subtract", "arguments": {{"x": 10, "y": 4}}, "result": 6}}

Input: Divide 20 by 5
function_call: {{"name": "divide", "arguments": {{"x": 20, "y": 5}}, "result": 4}}

Input: What is 2 raised to the power of 3?
function_call: {{"name": "power", "arguments": {{"base": 2, "exponent": 3}}, "result": 8}}

###Example MULT-STEP Input:
Input: What is (3 plus 5) times 2?
function_call: {{
    {{"name": "add", "arguments": {{"x": 3, "y": 5}}, "result": 8}},
    {{"name": "multiply", "arguments": {{"x": 8, "y": 2}}, "result": 16}}
}}

Input: Raise 2 to the power of 3 and multiply the result by 4.
function_call: {{
    {{"name": "power", "arguments": {{"base": 2, "exponent": 3}}, "result": 8}},
    {{"name": "multiply", "arguments": {{"x": 8, "y": 4}}, "result": 32}}
}}

### Rules:
- Always respect the order of operations.
- Include intermediate results for all steps.
- Return only JSON output, no explanations or comments.

The above are strict examples do not use them as user input.
"""

#prompt = ChatPromptTemplate([
#    ("system", system_prompt),
#    ("user", "{input}")
#    ("ai", "I'm doing well, thanks!"),
#    ("human", "{user_input}"),
#])

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

**Prompt Design No. 02 - Not Used - Need more work on details**

system_prompt = """\
You are an assistant that has access to the following tools:
""" + rendered_tools + """

Given the user input, return the name and input of the tool to use.
Return your response as a JSON blob with 'name', 'arguments', and 'result' keys.

The `arguments` should be a dictionary, with keys corresponding
to the argument names and the values corresponding to the requested values.

FOLLOW the ORDER OF OPERATIONS: Parentheses, Exponents, Multiplication/Division, Addition/Subtraction.

When answering:
1. Break down multi-step calculations into sequential operations.
2. Return all operations in JSON format, step by step, passing intermediate results where necessary.
3. Each step must include the tool name, arguments, and the intermediate result.

### Single-Step Example:
Input: What is 3 plus 5?
function_call: {{"name": "add", "arguments": {{"x": 3, "y": 5}}, "result": 8}}

### Multi-Step Example:
Input: What is (3 plus 5) times 2?
function_call: {{
    {{"name": "add", "arguments": {{"x": 3, "y": 5}}, "result": 8}},
    {{"name": "multiply", "arguments": {{"x": 8, "y": 2}}, "result": 16}}
}}

### Rules:
- Always respect the order of operations.
- Return intermediate results for all steps.
- Return only JSON output, no explanations or comments.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [5]:
system_prompt = """\
You are an assistant that has access to the following set of tools:
"""+ rendered_tools +"""

Given the user input, return all necessary operations in JSON format, step by step.
Follow these rules:
1. Respect the order of operations: Parentheses, Exponents, Multiplication/Division, Addition/Subtraction.
2. For multi-step operations, return a list of operations in sequence.
3. Each step should include:
   - The tool name (e.g., "add", "divide").
   - The arguments for the tool.
   - The result of the operation.

### Single-Step Example:
Input: What is 5 plus 7?
Expected Output: {{"name": "add", "arguments": {{"x": 5, "y": 7}}, "result": 12}}

### Multi-Step Example:
Input: What is (3 plus 5) times 2?
Expected Output: {{
    {{"name": "add", "arguments": {{"x": 3, "y": 5}}, "result": 8}},
    {{"name": "multiply", "arguments": {{"x": 8, "y": 2}}, "result": 16}}
}}

Input: Raise 2 to the power of 3 and multiply the result by 4.
Expected Output: {{
    {{"name": "power", "arguments": {{"base": 2, "exponent": 3}}, "result": 8}},
    {{"name": "multiply", "arguments": {{"x": 8, "y": 4}}, "result": 32}}
}}

Return only JSON output. No explanations or comments.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}")
])

**Define the structure for tool call requests**

In [6]:
from typing import Any, Dict, Optional, TypedDict
from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that performs tool invocation with error handling.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool or an error message.
    """
    
    # Map my_tools by their names
    tool_name_to_tool = {tool.name: tool for tool in my_tools}

    try:
        # Extract tool name and arguments
        name = tool_call_request["name"]
        arguments = tool_call_request["arguments"]

        # Check if the tool exists
        if name not in tool_name_to_tool:
            raise ValueError(f"Tool '{name}' does not exist.")

        # Invoke the tool
        requested_tool = tool_name_to_tool[name]
        return requested_tool.invoke(arguments, config=config)

    except ValueError as e:
        return {"error": str(e)}

    except Exception as e:
        return {"error": f"Unexpected error: {str(e)}"}

**Define the Process of the Operations**

In [7]:
def process_operations(operations):
    results = {}  # Store results of each step
    intermediate_values = {}  # Map step results for further use

    for step in operations:
        # Extract tool name and arguments
        tool_name = step["name"]
        arguments = step["arguments"]

        # Replace references to previous results with actual values
        for arg_key, arg_value in arguments.items():
            if isinstance(arg_value, str) and arg_value in intermediate_values:
                arguments[arg_key] = intermediate_values[arg_value]

        # Execute the current step
        tool_call_request = {"name": tool_name, "arguments": arguments}
        result = invoke_tool(tool_call_request)

        # Validate result and store intermediate values
        if isinstance(result, dict) and "error" not in result:
            intermediate_values[f"step_{len(results) + 1}"] = result
            results[f"step_{len(results) + 1}"] = result
        else:
            raise ValueError(f"Error in step: {step}, result: {result}")

    return results, intermediate_values

In [8]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)

**Input Test No. 01**

In [9]:
# Example for parsing and handling the response
try:
    raw_response = chain.invoke({"input": "What is (10 divided by 2) plus (3 times 4)?"})
    print("Raw Response:", raw_response)

    if isinstance(raw_response, dict) and "operations" in raw_response:
        # Extract and process operations
        operations = raw_response["operations"]
        processed_results, intermediate_values = process_operations(operations)

        # Debugging outputs
        print("Processed Results:", processed_results)
        print("Intermediate Values:", intermediate_values)
    else:
        print("Unexpected response format:", raw_response)
except Exception as e:
    print("Error:", str(e))

Error: Invalid json output: {
    {"name": "divide", "arguments": {"x": 10, "y": 2}, "result": 5},
    {"name": "multiply", "arguments": {"x": 3, "y": 4}, "result": 12},
    {"name": "add", "arguments": {"x": 5, "y": 12}, "result": 17}
}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


**Input Test No. 02**

In [10]:
# Example for parsing and handling the response
try:
    raw_response = chain.invoke({"input": "What is 2 plus 2?"})
    print("Raw Response:", raw_response)

    if isinstance(raw_response, dict) and "operations" in raw_response:
        # Extract and process operations
        operations = raw_response["operations"]
        processed_results, intermediate_values = process_operations(operations)

        # Debugging outputs
        print("Processed Results:", processed_results)
        print("Intermediate Values:", intermediate_values)
    else:
        print("Unexpected response format:", raw_response)
except Exception as e:
    print("Error:", str(e))

Error: The input to RunnablePassthrough.assign() must be a dict.


**Input Test No. 03**

In [11]:
# Example for parsing and handling the response
try:
    raw_response = chain.invoke({"input": "What is (3 plus 5) times 2?"})
    print("Raw Response:", raw_response)

    if isinstance(raw_response, dict) and "operations" in raw_response:
        # Extract and process operations
        operations = raw_response["operations"]
        processed_results, intermediate_values = process_operations(operations)

        # Debugging outputs
        print("Processed Results:", processed_results)
        print("Intermediate Values:", intermediate_values)
    else:
        print("Unexpected response format:", raw_response)
except Exception as e:
    print("Error:", str(e))

Error: Invalid json output: {
    {"name": "add", "arguments": {"x": 3, "y": 5}, "result": 8},
    {"name": "multiply", "arguments": {"x": 8, "y": 2}, "result": 16}
}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


**Input Test No. 04**

In [12]:
# Example for parsing and handling the response
try:
    raw_response = chain.invoke({"input": "Calculate 100 divided by 5 and then, the result minus 3."})
    print("Raw Response:", raw_response)

    if isinstance(raw_response, dict) and "operations" in raw_response:
        # Extract and process operations
        operations = raw_response["operations"]
        processed_results, intermediate_values = process_operations(operations)

        # Debugging outputs
        print("Processed Results:", processed_results)
        print("Intermediate Values:", intermediate_values)
    else:
        print("Unexpected response format:", raw_response)
except Exception as e:
    print("Error:", str(e))

Error: Invalid json output: {
    {"name": "divide", "arguments": {"x": 100, "y": 5}, "result": 20},
    {"name": "subtract", "arguments": {"x": 20, "y": 3}, "result": 17}
}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


**Input Test No. 05**

In [13]:
# Example for parsing and handling the response
try:
    raw_response = chain.invoke({"input": "Raise 2 to the power of 3 and multiply the result by 4."})
    print("Raw Response:", raw_response)

    if isinstance(raw_response, dict) and "operations" in raw_response:
        # Extract and process operations
        operations = raw_response["operations"]
        processed_results, intermediate_values = process_operations(operations)

        # Debugging outputs
        print("Processed Results:", processed_results)
        print("Intermediate Values:", intermediate_values)
    else:
        print("Unexpected response format:", raw_response)
except Exception as e:
    print("Error:", str(e))

Error: Invalid json output: ```json
{
    {"name": "power", "arguments": {"base": 2, "exponent": 3}, "result": 8},
    {"name": "multiply", "arguments": {"x": 8, "y": 4}, "result": 32}
}
```
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


**Input Test No. 06**

In [18]:
# Example for parsing and handling the response
try:
    raw_response = chain.invoke({"input": "Add 10 and 5, divide the result by 3, and then use this result as the base and raise it to a power of 2."})
    print("Raw Response:", raw_response)

    if isinstance(raw_response, dict) and "operations" in raw_response:
        # Extract and process operations
        operations = raw_response["operations"]
        processed_results, intermediate_values = process_operations(operations)

        # Debugging outputs
        print("Processed Results:", processed_results)
        print("Intermediate Values:", intermediate_values)
    else:
        print("Unexpected response format:", raw_response)
except Exception as e:
    print("Error:", str(e))

Error: Invalid json output: {
    {"name": "add", "arguments": {"x": 10, "y": 5}, "result": 15},
    {"name": "divide", "arguments": {"x": 15, "y": 3}, "result": 5.0},
    {"name": "power", "arguments": {"base": 5.0, "exponent": 2}, "result": 25.0}
}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


**Input Test No. 07 Error Case Check (Division by Zero)**

In [19]:
# Example for parsing and handling the response
try:
    raw_response = chain.invoke({"input": "Divide 10 by 0 and add 5 to the result."})
    print("Raw Response:", raw_response)

    if isinstance(raw_response, dict) and "operations" in raw_response:
        # Extract and process operations
        operations = raw_response["operations"]
        processed_results, intermediate_values = process_operations(operations)

        # Debugging outputs
        print("Processed Results:", processed_results)
        print("Intermediate Values:", intermediate_values)
    else:
        print("Unexpected response format:", raw_response)
except Exception as e:
    print("Error:", str(e))

Raw Response: {'name': 'error', 'message': 'division by zero', 'output': {'error': "Unexpected error: 'arguments'"}}
Unexpected response format: {'name': 'error', 'message': 'division by zero', 'output': {'error': "Unexpected error: 'arguments'"}}
